# Basic Configs

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [4]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 512
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [5]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 164 (delta 68), reused 86 (delta 29), pack-reused 0
Receiving objects: 100% (164/164), 1.58 MiB | 3.07 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [6]:
!mv IDL-Project project

# Load Functions

In [7]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [8]:
trainset, trainloader, testset, testloader = get_data()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load Models

In [9]:
from project.summarize.classifier import ResidualBlock, ResNet18, cifar_resnet20, ResNet50, VGG

In [10]:
resnet18 = ResNet18(ResidualBlock)
# With pretrained model
resnet20 = cifar_resnet20('cifar10')
resnet50 = ResNet50()
vgg16 = VGG('VGG16')

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar10-resnet20-30abc31d.pth" to /root/.cache/torch/hub/checkpoints/cifar10-resnet20-30abc31d.pth


### Load Trained Models

In [11]:
vgg16_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/vgg16.pth"
vgg16.load_state_dict(torch.load(vgg16_path)['model_state_dict'])

<All keys matched successfully>

In [12]:
models = {
    # 'resnet18': resnet18,
    'resnet20': resnet20,
    # 'resnet50': resnet50,
    'vgg16': vgg16,
}

# Generate the Adversarial Samples

In [13]:
from project.summarize.generate_attack import generate_fgsm_attack

In [14]:
criterion = nn.CrossEntropyLoss()

In [16]:
adv_samples = {}
eps = 0.05

for model_name, model in models.items():
    model.to(device)
    model.eval()
    acc, adv_ex, adv_label = generate_fgsm_attack(model, criterion, testloader, testset, eps, device)
    model.to(cpu)
    torch.cuda.empty_cache()
    dataset = AdvDataset(adv_ex, adv_label)
    loader = data.DataLoader(dataset, batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)
    adv_samples[model_name] = (dataset, loader)

Epsilon: 0.05	Test Accuracy = 4627 / 10000 = 0.4627
Epsilon: 0.05	Test Accuracy = 4320 / 10000 = 0.432


# 4x4 Matrix

In [17]:
from project.summarize.normal_train import evaluate

In [18]:
dataset, loader = adv_samples['vgg16']
print(dataset[0][1].type())

torch.LongTensor


In [19]:
matrix = {}
for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, (dataset, loader) in adv_samples.items():
        print('classifier: ', model_name, ';  adversarial data comes from: ', adv_name)
        loss, acc = evaluate(model, loader, criterion, device)
        line[adv_name] = acc
        print('acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  resnet20 ;  adversarial data comes from:  resnet20
acc =  46.27
classifier:  resnet20 ;  adversarial data comes from:  vgg16
acc =  65.16999999999999
classifier:  vgg16 ;  adversarial data comes from:  resnet20
acc =  73.76
classifier:  vgg16 ;  adversarial data comes from:  vgg16
acc =  43.2
